In [1]:
# Modules
import sys
from time import process_time
import os

import verbose_module
sys.path.append('../../.')
import pIRPgym

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


In [2]:
########################################## Instance generator ###########################################
# Instance Generator
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

env_config = {  'M':13,'K':15,'T':12,'F':13,'Q':2000,
                'S':6,'LA_horizon':4,
                'd_max':2000,'hist_window':60,
                'back_o_cost':10000
            }

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)


######################################### Environment ##########################################
# Environment
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

''' Parameters '''
verbose = True
show_gap = True
instances = dict()
instances['Li'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()

In [3]:
if verbose: verbose_module.routing_instances.print_head(show_gap)

for ss,inst_list in instances.items():
    if ss == 'Golden': print('\n')
    for instance in inst_list:
        # Upload dCVRP instance
        purchase,benchmark = inst_gen.upload_CVRP_instance(ss, instance)

        if verbose: string = verbose_module.routing_instances.print_inst(ss,instance,benchmark[0],benchmark[1])

        nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
        RCL_routes,RCL_obj,RCL_info,RCL_time  = pIRPgym.Routing.RCL_Heuristic(purchase,inst_gen,env.t,RCL_alpha=0.001)                                 # RCL based constructive
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    RCL_obj,len(RCL_routes),RCL_time,show_gap,benchmark)
        GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    GA_obj,len(GA_routes),GA_time,show_gap,benchmark)
        HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)
        # CG_routes, CG_distances, CG_loads, CG_time = pIRPgym.Routing.ColumnGeneration(purchase,inst_gen,env.t,verbose=True)       # Column Generation algorithm                  
        # if verbose: string = verbose_module.print_routing_update(string,sum(CG_distances),len(CG_routes),end=True)                                        # Column Generation algorithm

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------
Li 21 	| 10 	16213 	| 0.06 	  13	28.0 	| 0.28 	  14	33.22 	| 28.4 	  13	21.1 	| 30.1 	  10	0.06 	|
Li 22 	| 15 	14499 	| 0.06 	  22	31.72 	| 0.27 	  23	38.69 	| 53.3 	  22	31.22 	| 30.2 	  14	-2.61 	|
Li 23 	| 10 	18801 	| 0.07 	  14	41.85 	| 0.28 	  13	29.66 	| 62.8 	  12	21.99 	| 30.2 	  10	0.12 	|
Li 24 	| 10 	21389 	| 0.09 	  13	30.59 	| 0.39 	  14	34.54 	| 70.5 	  13	24.51 	| 30.4 	  10	0.9 	|
Li 25 	| 19 	16666 	| 0.12 	  28	31.56 	| 0.49 	  31	42.97 	| 71.5 	  28	31.77 	| 30.3 	  18	-5.37 	|
Li 26 	| 10 	23978 	| 0.13 	  14	27.61 	| 0.53 	  14	29.63 	| 93.9 	  13	20.55 	| 30.5 	  10	1.86